In [ ]:
import pickle
from datetime import date

import matplotlib.pyplot as plt
import polars as pl

In [ ]:
## data ##
data = pl.read_parquet("../output/flu_state/data.parquet")

In [ ]:
with open("../output/flu_state/fits/fit_2022-04-01.pkl", "rb") as f:
    fits = pickle.load(f)

fit = fits[("LPLModel", date(2022, 4, 1))]
pred = pl.read_parquet(
    "../output/flu_state/pred/forecast_date=2022-04-01/part-0.parquet"
)

In [ ]:
all = pred.join(data, on=["season", "geography", "time_end"]).rename(
    {"estimate_right": "data", "estimate": "pred"}
)

In [ ]:
sse = all.group_by(["season", "geography", "sample_id"]).agg(
    ((pl.col("data") - pl.col("pred")) ** 2).sum().alias("sse")
)

In [ ]:
plt.hist(sse["sse"].to_numpy(), bins=500)